# WBC and SGB Policy Portal Sraper
The policies are stored behind a loging page, can be multiple pages deep,


In [1]:
from  scraper import WebScraper
from loguru import logger
import os

class WBC_SGB(WebScraper):
    def __init__(self, domain:str, cookie:str, download_path:str, search_only:bool=False):
        super().__init__(domain, cookie, download_path, class_names = ["div#wrapper"])
        self.search_only = search_only

    def scrape(self, url:str, depth:int, starting_page:bool = True, ) -> None:

        logger.info(f"Scraping {depth}: {url}")
        if self.skip(url, depth, ['/nlpsearch/']):
            return
        
        _, extension = self.get_path(url)
        assert extension in [".html"], f"Invalid extension '{extension}' for {url}"

        # download unless except starting page
        html_content = self.save_url(url, starting_page)
        
        # extract all the links and recurse
        self.extract(html_content, depth, "div.rn_Container" ) # "div." or "div.rn_Container" or div#content
        
        return

# St Georges

In [4]:
import shutil

directory_path=r"/home/kevinmcisaac/Projects/policy-pal-pages/policy/SGB/Credit_Risk_Policies/"

if os.path.exists(directory_path):
    shutil.rmtree(directory_path)  
os.makedirs(directory_path) 

#Need depth 3 to get to 
sgb = WBC_SGB(r"https://creditriskpolicies.westpac.com.au", 
              cookie=r"fpid=bf332293-c15f-4a64-8750-4af5a6a25b05; TS01bae890=01a3dc9c21fea561826e1415ef3c74fb9df9744f5c9f8eb9011a5a6256dd3908d4f1a937553e86574ab2c3b2993fb43d8de5148510; aam_fb_ads=fb_ads%3D22915918%3B22915983%3B22969655%3B24893962; aam_gads=googads%3D16500962%3B24893962; aam_sem_gads=googsemads%3D22969655%3B24893962; AMCV_3A4B7BAF56F01DA67F000101%40AdobeOrg=MCMID|20354680758413087533313270027196719041; _gcl_au=1.1.1748561074.1735870594; __qca=P0-750036038-1735870593993; _ga=GA1.1.1313640549.1735870595; LPVID=Q5YjI2ODFlMzZkZWNjMzgy; aam_edge_seg=segment%3D12324502%2Csegment%3D19586959%2Csegment%3D22915918%2Csegment%3D19586952%2Csegment%3D22969655%2Csegment%3D13955774%2Csegment%3D12324503%2Csegment%3D18159756%2Csegment%3D22915983%2Csegment%3D24893962%2Csegment%3D11996985%2Csegment%3D11996985; kndctr_3A4B7BAF56F01DA67F000101_AdobeOrg_identity=CiYyMDM1NDY4MDc1ODQxMzA4NzUzMzMxMzI3MDAyNzE5NjcxOTA0MVITCLz6zM%5FCMhABGAEqBEFVUzMwAPAB3ZWcy8Qy; kndctr_3A4B7BAF56F01DA67F000101_AdobeOrg_cluster=aus3; _ga_WQCDREZ4DH=GS1.1.1736398276.8.0.1736398276.0.0.0; s_ecid=MCMID%7C20354680758413087533313270027196719041; LPSID-16303569=ADk71cszS0CZOkETYON_7w; cp_profile=eUnSy1lsA7K8bshsuhLHRR9Sno6S7pUIR~ruw_5HHfr7EW1nnFEWPhL6PdGNS9EQMrg7Xdci5Q4vKfQg78MtARF09slToL0Lcg1HfVwfobySISlWBw9b2vqJlbhGHC4Wvp~NozCmWHrwB1v2cccDgIPefL2isisShDwjAKpPA3Ss_V78IDbflpFw!!; policyType=MORTGAGE_BROKING_OBP_SGB_GROUP; cp_session=fUblpFpYKMNpZmPhr9BZNRReKDCEMfB5SEUSZ8zEJgyl4H6LUzveH0gnP3SYBXAm~C2FIWRSKrarpTgwWDEu_jtb1t6zwMmLGfDDxEH~H2gfWPpJhW5FxyxUb8OEaARek2O6gJRYmgj65MorugEqovpIo6BZbiyJnfhdzLU9CK0jKom73N6CoWdldgm9PYyxE5RELghZmSAxKWecLcCPwsr~dhQydPy4SNg4J5JjGFUeeN2ystp2ls5wU9CqIx0rXRh5pLSbF1gAvznDTOIVNBMMOTpHvuilmMCP~mnNc1lwvpvLKhG_KyTjBKESkiF6TewLbYyDxWUAK36ms79aq4UJOPeVViSSNExe0ggaOy5Z0E2z0e_2wpluAJDPoJ2zrBd2x5h18JG8VI3_pgVVVtdYRZceNMZYV9mq~TQCzwA28Tadntjr09kS0vBxAKGHbg4w6womV8H4ZahPGLABY19WoZ5ITlcojEshO9hd4m9hlPH7PYkpB280d0AV0riet_Uuynk2uooaks15xspUv_dlz4EMd9JGXOfcZH~mDvvBJwZgW8LkjgEJpmnWtFO1KdL4RLbfTEZ2dEzFy3QKuWT5yL~ePRla5R4HJNXGZc~FlvYFGpSJanc7bJXKfQ0FGsxR9_9lSKOd0Th2fBIQT5f080GKzd0lYESNzHE5wFHhL799ArVUP4VG51lCD9e_14am0Sb_FrG4dJCXlfhMq8gwEqO46vXhh~cdSKC7vGMF72j2EdOkYt7UVkPouh626b2USaNPZTTiFxtK2i5WH2GamfG177wnJqMQvj6wq7GVWFkQVx5SdaCmIff2gUyypOEgeeiSA02IjF0eaW0u8qBYAcSw9WUQ5GVtCbqthVFJBs3LKee15h42J2Ihd8brHFheejbrN_wN4Jj2pknALLqKPp6DYwaDv1ghN8EIcoLXuwCI3GHjqMFxbAFEEf0AAQC11cvUDgtzjzdA9EdsGfVu0wgroyHpkFtvySV3EyZp0gpz6WO6ILHXP~6Fc7sLfCawzKPPY88nC3LBrOTQF8~NhFbHQffPNCby1DtW7UtJLahUn2BoiieEYPlt7dmLLvQX5dVFYH_HQ3pv8lv02klMRxdw0VWoJ~n50jeOJtj3f43lv9QtM5HPhtab4OKTE3_aCAOxyy~AMXeRX3LseHqqsJV1wY3CIYzZlpH0kt02~MRm33rJgX1JkhgjAVMsMDIcbNNCmVkvJ~t1fwur0BWR7UO6SGhi2LSefKd4QNdUI4r6ARsF7Ag2eQjST3on7a6iff__D40hgZDKlAGo1mo_SrUcczRlXExyyr5P6JXAX6QiRsVoLJHw1t8E_eT0cs_N7oO9hagrrIaQHktkGisy3Am5d5Nx6VQezIzZie4mhg_1rNEFW341DcChXyGMZ9ivhCmGXYoYDz~vCMsX24eyFINu0OLNEIhcMFbQaOr1dEjI3UD2XkwgeuXSlJk0AFVdTWRkFtSFgVjMuNO55JEg9FlzNpTfNSrbIu3XEEBQojWN7r2QHx1JXCX7acOsAxoChYtlc3eirqUXvl9S1ouaMRL2sOSiMNM0l2B39RWLQrwxb0w~A5UierexWHWQYR4CLiugyQYOfsDyXZxcx4IfBMA7wMtKgHw4w94RIVc2MEJAOWgOAllZ_noyfUXwSE7dfx4lIzwH4gWeGj1f6imQntXKkkDg7~UY03pNR2ZsTT6Cyxh6jPmyaGufh8soaa1AmtlWlGw5lCF4rx6kzcJSU1DA2~M54~5mYe7y6Kr7sYq~hCOuTUsQBlF~~qekwCmqSxja4Zw8ejh0UlKipbD19PKtbEDgNE8B~OSdaenjQqmmMN~tVRCyQdrac6FIhJvux~tqFyA7sSAB0Ymzg~qCEvtmDqN~ZcIz5qWZ_aWQzt8BIFl94w0FRQSRwz6PQTdibQZQTTVOtex2_h7EbKDLFn4R64bVpnNn1b6g2QUkkkUZ71DzytMw_0Xh9TCII~x4e1qzT_QEl0d5rwV1X1GjG3~5P1EpL6n3Pg9zs3hLvOapoL_1KwPMmEF~8mXVgAUvAfa9QTldLx_UOLxzzLNli9KJK6tyKkH~ra3dv0EnhNYg4CN9YpAmoHpCubaSu5s~mA5qecXv6VpSE1dmMj486RVCnTMmWJ8kkgQxALyp1_3RQ7HI6iz3hI6bwUkHG7xaKFIIesBkB2NL6d90zrq1vQDw_k635Y9CA4S3lI3_ZNDItQsC6chOglbpl2x55G~NAC2s_DCwrzEd9I5KR_5mKHGdQ_W5gSq0gszN4uRbJKpBxYNWcMxVf6FFBNPqDr_8dK2oqPyrz_cbFuOsLmVnnHgN9ToLcejUc7TYFy2DnSj5Z4Z8cx4cRam90TY7CN6ZwNHvRzhpZECqzvhw5sGBdOh2jWMVNF8H6Mir6lvytkCaMDgxPdjApI8u1q62aSaxuFJsEDByZgwSqmwxkATBQyt7jA7pLey2zgZuAgyd5kHA6BuarcltAwaCpZsMz1iNn4nwRHpdV5lhX6iMuTw93v9kvPz4bQQu8vxJTah~OResmTUhZWqJitlM8tTctFdfzEqAD~pZoAA4pQRyKUbsUiX7IewXkXeZtPHd7HvJnxTOsNkcE9jk7uUyzgT_2g6Xw9Ac3sNPJOw50p9XCwQxd_Za_wrLIIM2FeM9V~do5kUzPAHoWBBOF~hGDnIodo65DCKzyXX4I2NmScQOhRoic0k2PJXqY1HLfMpg2jFbaKgL2_IBCfbUbfOsPQ12oHiv3_khoKb2_6rekhxHYjFx4F3~uuovgohJKMl0Abb5tLPr9kvR54bjPem3zpV7G89TX1oBLaO4YoKJEDfm5IN1RhF112E4UT4M6j8a1cDn~DKvbeowgtXYZRLceibdTeuxjXJ0GrOswXBFyceeFOOCQKMflZLCoUHY9dvPFvmR4eQThGK5frvsuIm4sY0ilkJ5x6bHyqUX7CCevVpaOvrZBletN8cPUodaSa2IcHDvu0_0CvDr1~GKABZXtGdXt56PLlIKptS~Grn9FQ6JfVm0dFo2U6q5507M_Zw~gNwDiUptaJgQnz6cEBqoZEYozDBi~e16Z8jaqkju45ONY0cm8Kdkk_EYF6h42OtIlqJgwd7RKTzXd_jlz~gGIVvVvVcTv8VPYoS2rAgvhwmVwub8iYV0nkprMubUirLIsx5ykfa6PlrYH9WhwCjymddJ5b1gtlqMfxMqIJ849vhviUPSg_AJJ~T2xlaxZvQmcgaSaHD8i7duGgJSwEhyEF6MEP2mcOTpGCzTuRkfrrnCS0HZEviBz9JZ6Y9VSCVnUvf7HvwhC_3w3FrLYkxipqsq5lFPDFBpcG_Hn0kmM02EpPycUbJKca6rC7rfTjINSSUYEWW2L6IF2zpkaFfDV_1hE~w7qR9Zfz2UK9bUsy5lg1DzioqvGwPXdPPWnEtKZ6ZCaXfj2aje1varp~sa7aIIFa1SGt_i4VNDdRZU6pjj_JC8XWMo4OWyyFkf5Ea8JEgQUHPDpftBKk5Z4Lq1KnTamqlN6xE9ic6iIsdpR~NVLYnZNsTIVWfsmfZ8KfgTWv_4Y~mZYjfu1k8Uugpfn40PdNLYibovaHErBfKMxKDnORRzTx9NfMdw9oNpKEolEowTaQyt1U_Vwq3SuWyYd58tdHlJBrF2OJyPRo9p2Djk14ZastvO2ePmbbI8EDH3UrVU6uNHa8KdJYewbidaP4WeuF4hwVtRPQwEv8jPMjuMtbO6~BWq64RCIx4VurMzZ3cymXJudpOP21JO_ddXEuZMzh3A2tlKtN_UKyTYg5kRhYUOa91oDIPVIsoJJ7U6MX4DmNxZqyLPWHBl6JaBopl05dfcOcuU4dPFXTox9vyJi8~rwhBmXRROD~K4WxaUR29pFQXOfd1EWVXn_FaS4dHV0Qwb06Kjqr9dGjiahGhZFk7eXeN8zwo6H_XAK2H1e~bw69auFCPWrHsw!!; ADRUM=s=1736399094839&r=https%3A%2F%2Fcreditriskpolicies.westpac.com.au%2Fapp%2Fbrowse%2FchannelRecordID%2FMORTGAGE_BROKING_OBP_SGB_GROUP%2Fa_status%2Fpublished%2Ftype%2Fbrowse", 
              download_path= directory_path, 
              search_only=False)
sgb.scrape("https://creditriskpolicies.westpac.com.au/app/browse/channelRecordID/MORTGAGE_BROKING_OBP_SGB_GROUP", depth=16)

print(len(sgb.download))

2025-01-09 13:05:33.748 | INFO     | __main__:scrape:12 - Scraping 16: https://creditriskpolicies.westpac.com.au/app/browse/channelRecordID/MORTGAGE_BROKING_OBP_SGB_GROUP
2025-01-09 13:05:34.665 | INFO     | __main__:scrape:12 - Scraping 15: https://creditriskpolicies.westpac.com.au/app/answers/answer_view/a_id/1000681
2025-01-09 13:05:39.422 | INFO     | scraper:save_url:62 - Downloading https://creditriskpolicies.westpac.com.au/app/answers/answer_view/a_id/1000681 to /home/kevinmcisaac/Projects/policy-pal-pages/policy/SGB/Credit_Risk_Policies//app/answers/answer_view/a_id/1000681.html
2025-01-09 13:05:39.490 | INFO     | __main__:scrape:12 - Scraping 14: https://creditriskpolicies.westpac.com.au/app/results/kw/BOS1/nlpsearch/true/loc/en-GB
2025-01-09 13:05:39.491 | INFO     | scraper:skip:134 - Skipping excluded /nlpsearch/
2025-01-09 13:05:39.491 | INFO     | __main__:scrape:12 - Scraping 14: https://creditriskpolicies.westpac.com.au/app/answers/answer_view/a_id/1000726
2025-01-09 1

103


# Westpac

In [2]:
import shutil

directory_path=r"/home/kevinmcisaac/Projects/policy-pal-pages/policy/WBC/Credit_Risk_Policies/"

if os.path.exists(directory_path):
    shutil.rmtree(directory_path)  
os.makedirs(directory_path) 

wbc = WBC_SGB(r"https://creditriskpolicies.westpac.com.au", 
              cookie=r"fpid=bf332293-c15f-4a64-8750-4af5a6a25b05; TS01bae890=01a3dc9c21fea561826e1415ef3c74fb9df9744f5c9f8eb9011a5a6256dd3908d4f1a937553e86574ab2c3b2993fb43d8de5148510; aam_fb_ads=fb_ads%3D22915918%3B22915983%3B22969655%3B24893962; aam_gads=googads%3D16500962%3B24893962; aam_sem_gads=googsemads%3D22969655%3B24893962; AMCV_3A4B7BAF56F01DA67F000101%40AdobeOrg=MCMID|20354680758413087533313270027196719041; _gcl_au=1.1.1748561074.1735870594; __qca=P0-750036038-1735870593993; _ga=GA1.1.1313640549.1735870595; LPVID=Q5YjI2ODFlMzZkZWNjMzgy; aam_edge_seg=segment%3D12324502%2Csegment%3D19586959%2Csegment%3D22915918%2Csegment%3D19586952%2Csegment%3D22969655%2Csegment%3D13955774%2Csegment%3D12324503%2Csegment%3D18159756%2Csegment%3D22915983%2Csegment%3D24893962%2Csegment%3D11996985%2Csegment%3D11996985; kndctr_3A4B7BAF56F01DA67F000101_AdobeOrg_identity=CiYyMDM1NDY4MDc1ODQxMzA4NzUzMzMxMzI3MDAyNzE5NjcxOTA0MVITCLz6zM%5FCMhABGAEqBEFVUzMwAPABqbacycMy; s_ecid=MCMID%7C20354680758413087533313270027196719041; LPSID-16303569=yM2VMtuFQZytQi7QpMZ1HQ; kndctr_3A4B7BAF56F01DA67F000101_AdobeOrg_cluster=aus3; _ga_WQCDREZ4DH=GS1.1.1736134093.6.0.1736134093.0.0.0; cp_profile=eUuzhJsrR6EgsmAgUh8zZi5PXBkPe19uHyR1cVTCanpdTLn1RgrnTPitFsd0AUg133oVEzjlcrTVgrcHz3XRi1tqGnX7fshKkVFtiS3oNqD0zpBcDvXxDqyFeV_XbHZhnY4j6gLFVWtzHcP18HtwZhelAIdzlz8tc9; policyType=MORTGAGE_BROKING_OBP_WESTPAC; cp_session=fUjX7mVUS7vjVcnZB9GNXlVV9KpwqtDf3il8W2YIowYB0uKSuqspnyq8hLegboytUpC5oxAeLBg4MNrNHPrmYJynA4kWf~Nc2w~LYa1XdGX2RREoGpabaa6QuplvJa9Txr3rX5E67zevVzBO0LVKmG8hLngdb6VnZH0F5Ow6FS2ar5FU92i9rcrLJ5nsEZ8p~duSBzhLysUoUrtAea6cO8j5Vu0eiOt9iEmVtAaYYIp0bJoh~HZKcfUnHIJTdi2ectaRDN89aQrOtcKo84Y5f_H5LfSobqo5efDLn0oJlD_H89WWcTS6qcU8kbHehysPZ7exjKJpi3qAACq7EC5ybIL0dt~Wpr4W~Ia3EtQoxqtB3cUDDRbsSOdV8s8pgVzoYjwtdO8uzTLouOSDiOTets2GHXwI3EfcFrTIih9_UG2vvEDX5Nn4pNR1K4kwGd5xgdh__vUJhADss0mWFh0bP4zoiOluhzIIg8wK5fmYGr9_jATiHy6xS2nrPw5DJoeXoHXwMP~MYuX6RXpycNvVVqcwqbNTMnnq_gqrCD3_FklcmLK_kmBfGgDbqeoNcWp4IyljK62txf5LcCo9XEHTV3oez8FqBTUzfR8edEVD3Vs9Dk3UHzk~gab~xiw9y2AYn0WjnRYMm7R9tTOp7rtjF4lE04apISqEmAkgPca~StthUjtdJLKj3uuoMgj~4nwQ~_h62eK6R~DpiRpXnNgA2vEBe0LQcsC6~R92RIv_wfpAr59cHfAaGKkcVBwhA~3a8JwGg2UVASrTxjFz0qM5dYvbLMZ05SD6_JJfxMshfEjxUWawx9AZcxu79SGOBg6IvZTnJ05fn47lJEeORPNDYBjoWKmDyUx0DJy4nEvO7qe__L0nJIidiwG5lAqIoZVryHMzEDqQVtO44K9TIYVn8vDq3GlZjE5IvqjN_Zt2_MUxD8xojmwMmPF9zR7aUE7CJlo~K~s4GkIEx13XdIS0tgYdN2yOzs4gb2x9VU5_5gpBKv8O1uu1f2YiltVpQrGZsivbNUuJoWHKfpffXzQ71OgiQZDopBXJfqhvY8t8o9KIlYxYzHW_4i6_~3vXv~n73c7o_x_7f~bddFqh4Dw32~OgdsPgzv~IxQDBWEwqjzLTaXRjNTojIZNfLbjupfVL0ds15kC_Q9Rw~FIeR4JB9JYypIQR0rT1oqnC5d_eDxcNcaNV~TxFJyy09dF2oKgqZ2hUjyPKifS4tQIXC5dQ7pi_hxsx5Zi9ClgU70ssvyqdGagTXzfLQwxnfCSAavpNbvBqkZ80cxgledM2sug2_GBJz_iznAkD6qWWAIBILQQ9~On76OAAE4G2yHi~sATOAyDlzoLH6Cs4~4vH9o5UgdNA_JBqotLLFqNESx_malF5wLAXT~P4qOyct2s6LskQLwVp1ORStzBTyVS2lo7QKBmWQuvSYNlaQwYvPMGLOragteJXHgeV8HLdTENqkZPnTXKn974VqmOpmeLMKKw4rBS2AcM6_U5A1MLqug1o8RWmFX11p60JKogzwvLwNrIAReaqSvhmFjkJKOx3x_U_TT_zM~QjWkXySQU2JdlNc4AVPKJAL2s521b4vnohdsM9GMVO_wdCM_v2q96AA2Lau7OUaA4L3eVtW5zsyEZfXRCBm5wYbF2te7g9omiFfLRt51Thv0RH42KgYZ_NULSinIc_8Zo6swsqlHdn8luvYwV4VbVi5nfBm2ETzu5_T7BGimHj09Ks9PtAldV0wX1ufXNvZU~m1hJ3e8uTQZo7L584AuteZYio1id51zT_LRrjqVpiwzSKNXKDHYnueC7o3J_88bNGuBcAQXkpho0cFn9KJg0_6A_XKofxg8j~BzRoDOJekjDloud96K732qBtIisfRcoGXmiwfIBIyVnKNbl9JJ7vLxGGaKeEHA9gsE1H5jg0biBL4FooM68AzVWWNViBS3BBoHjEQrL_FcfL61W87blu_nItsS3e66tvQSSHgARvYGptJ3Ahm_lJa~O0I2HrsdYdgzhlNbgFrdw41cz7t8Br3cTFDZWXMHFwCgABHRvS2znTi48T9Jzu0kvci6q7XGVlXLnNzdVelVUKFmPSqsgz3Mg89G4vSRW~dXNuT_fPBO0No9QjHaCoxbnJ_tvb7x4WSEilhXp_c2lGFVrnPZcLZi~mry7LauQDxzwW4xUH7HHUUKK_KN_WNowjSZspGcn16e95jmnfjZPwgv4FfbuqN3JSkuFDH~dMIhyusGTdB0qCv6d3BlYRFO0ZHRESBv94dnhp5Mgt_DInnBS5z3nl8kl3Vgizo3CsA7vBNPjBRKBFoiJnfn43mSp8WY4jwvDrg9afZQwmgudTSln_FVHDv8idPh790Ud_bDHxBDH02VHREtmkTB3ZKovMzwwKkADUwWjSJPzNukVQG96ZQ7m4Ffhw28vsenDUxJSqp0XUnmINXEesUMAMcrv0G_7ejB~sYWFQtYBQTILlNNPeXA1lESUVsBCkHp~of9S_uTRpoPjMJ6YVQ2DxWRuhsFdPkd7jtUJzYc2HMGEm5~VdyI3BfoywSy0sBaSiag469ibvA87kOn~ceg8lJbq47ixyQvyBG0esLCmI9saJ6c7OPb6JtDXZv~1HenxJVQp54XpvV_JNry6DneU9iJb3a3ssqOTLsw98MLA7N7rTzDd33Fesjln8elnl~yGRixuGGvsDJ0osLU8in2sZ3pSTU6gkAeeqRQusRKHd6gzIAOUmhJgpgp3JyjIsHPpjQ2t1dbLyilxuhofdiHk0JTXX8wYT633ey5WIn~Y9iWYxympgQ6HlDCECfPtZyX5ErJOSyHOPg1DsagcjgZlKyDE82kZ03SbCSw~DUBcxouUztfGe26AD2XeyAMucxDZ__jRaJ~ciKCgAxaRmtxfK7ntaIjMwZskK8Y8MTjF_cKfZMkl_mqi1_9bTiUQle0fya9HOyMUbu4FfNvC~R0yDgPqc3GRf_YW5~FilRoGsz2Ba2CjxDMlQch1xKeVIeqk3YuF~IGpQ52KD81YMugj~_~DbvWiiim5dA9qrPJI~DEMWUpVCxT8DOjDqX0CP1jyApkB0M0D0xNnZHhADLfNwbgwlENa~WfQQaG6Pyb7UyeAbX6FDuJV9W19lJ3QQ!!; ADRUM=s=1736134365930&r=https%3A%2F%2Fcreditriskpolicies.westpac.com.au%2Fapp%2Fbrowse%2FchannelRecordID%2FMORTGAGE_BROKING_OBP_WESTPAC%2Fa_status%2Fpublished%2Ftype%2Fbrowse", 
              download_path= directory_path, 
              search_only=False)
wbc.scrape("https://creditriskpolicies.westpac.com.au/app/browse/channelRecordID/MORTGAGE_BROKING_OBP_WESTPAC", depth=16)

print(len(wbc.download))

2025-01-09 12:59:06.306 | INFO     | __main__:scrape:12 - Scraping 16: https://creditriskpolicies.westpac.com.au/app/browse/channelRecordID/MORTGAGE_BROKING_OBP_WESTPAC
2025-01-09 12:59:07.599 | INFO     | __main__:scrape:12 - Scraping 15: https://creditriskpolicies.westpac.com.au/app/answers/answer_view/a_id/1000680
2025-01-09 12:59:12.677 | INFO     | scraper:save_url:62 - Downloading https://creditriskpolicies.westpac.com.au/app/answers/answer_view/a_id/1000680 to /home/kevinmcisaac/Projects/policy-pal-pages/policy/WBC/Credit_Risk_Policies//app/answers/answer_view/a_id/1000680.html
2025-01-09 12:59:12.750 | INFO     | __main__:scrape:12 - Scraping 14: https://creditriskpolicies.westpac.com.au/app/results/kw/BOW1/nlpsearch/true/loc/en-GB
2025-01-09 12:59:12.751 | INFO     | scraper:skip:134 - Skipping excluded /nlpsearch/
2025-01-09 12:59:12.752 | INFO     | __main__:scrape:12 - Scraping 14: https://creditriskpolicies.westpac.com.au/app/answers/answer_view/a_id/1000725
2025-01-09 12:

55


In [ ]:
import magic
import glob

directory_path=r"/home/kevinmcisaac/Projects/policy-pal-pages/policy/WBC/Credit_Risk_Policies/"
files_ext = glob.glob(f"{directory_path}/**/*.html", recursive=True)
files = [file for file in files_ext if magic.from_file(file, mime=True) != "text/html" ]
print(len(files))
files